# SIR-Modell

## Parameter

- `a`: Die Übertragbarkeit des Virus. Hängt davon ab wie viele `Suseptible` und wie viele `Infected` es gibt.
- `b`: Die Rate wie schnell man nach einer Infektion zurück zu den `Suseptible` geht. Das Individuum hat also durch die Infektion keine Immunität erhalten. Hängt davon ab wie viele `Infected` es gibt. Reduziert die Anzahl `Infected`.
- `c`: Die Rate mit der infizierte Individuun *heilen*, also sich immunisieren. Hängt von `Infected` ab. Reduziert die Anzahl `Infected`.
- `d`: Die Rate mit der geimpft wird. Hängt nur von `Suseptible` ab. Reduziert `Suseptible` und erhöht `Vaccinated`.
- `e`: Die Rate mit der immunisierte (`Removed`) die Immunität verlieren. Hängt von `Removed` ab. Erhöht `Suseptible`.
- `f`: Die Rate mit der geimpfte Individuuen den Impfschutz verlieren. Hängt von `Vaccinated` ab und erhöht `Suseptible`.
- `g`: Die Rate mit der sich geimpfte trozdem anstecken können. Die Idee hier ist das sich geimpfte anstecken können, aber nicht mehr sterben an der Infektion. Diese Rate hängt von `Vaccinated` und `Infected` ab. `g` sollte sehr klein gewählt werden, da er von 2 Grössen abhängt.
- `h`: Die Rate mit der infizierte-geimpfte Individuuen von `Suseptible` anstecken können.
- `i`: Die Rate mit der *False-Negative* getestet wird.
- `i`: Die Rate mit der *False-Negative* getestete Individuuen andere aus der Gruppe `Suseptible` anstecken.
- `k`: Todesrate bei Infizierten. Hängt von `Infected` ab und erhöht `Dead`.

## Events

Mit dem Objekt `events` kann das geschehen der Pandemie kontinuierlich verändert werden. Es kann für jeden Zeitschritt `t` ein Event angegeben werden. Bei einem Event werden die Parameter der laufenden Pandemie neu gesetzt.

**Beispiel:**
Zum Zeitpunkt `t=50` werden Massnahmen umgesetzt, welche die Verbreitung des Virus erschweren.

```python
events[50] = {'a': 0.05}
```

In [1]:
import matplotlib.pyplot as plt

from lib.iteration import *
import ipywidgets as widgets

plt.rcParams["figure.figsize"] = (15, 30)

a = widgets.FloatSlider(0.5, description='a:', step=0.05, min=0.0, max=1, readout_format='.2f')
b = widgets.FloatSlider(0.1, description='b:', step=0.05, min=0.0, max=1, readout_format='.2f')
c = widgets.FloatSlider(0.1, description='c:', step=0.05, min=0.0, max=1, readout_format='.2f')
d = widgets.FloatSlider(0.0, description='d:', step=0.05, min=0.0, max=1, readout_format='.2f')
e = widgets.FloatSlider(0.0, description='e:', step=0.05, min=0.0, max=1, readout_format='.2f')
f = widgets.FloatSlider(0.0, description='f:', step=0.05, min=0.0, max=1, readout_format='.2f')
g = widgets.FloatSlider(0.0, description='g:', step=0.05, min=0.0, max=1, readout_format='.2f')
h = widgets.FloatSlider(0.0, description='h:', step=0.05, min=0.0, max=1, readout_format='.2f')
i = widgets.FloatSlider(0.0, description='i:', step=0.05, min=0.0, max=1, readout_format='.2f')
j = widgets.FloatSlider(0.0, description='j:', step=0.05, min=0.0, max=1, readout_format='.2f')
k = widgets.FloatSlider(0.0, description='k:', step=0.05, min=0.0, max=1, readout_format='.2f')


s_start = widgets.IntSlider(10000, description='S:', step=100, min=0, max=10_000_000)
i_start = widgets.IntSlider(10, description='I:', step=100, min=0, max=10_000_000)

days = widgets.IntSlider(365, description='Anzahl Tage', min=1, max=730)

daily_fluctuations = widgets.FloatSlider(0.0, description='Tägliche Unsicherheit', min=0.0, max=1.0)

def update(args):
    fig_panel.clear_output()
    with fig_panel:
        l = iteration(sir_modell, x0=[s_start.value, i_start.value, 0, 0, 0, 0, 0],
              n=days.value, a=a.value, b=b.value, c=c.value, d=d.value,
              e=e.value, f=f.value, g=g.value, h=h.value, i=i.value, j=j.value, k=k.value,
              daily_fluctuations=daily_fluctuations.value,
              events=events)
        to_plot_labels={l.description: l.value for l in selectors}
        pl = plot_iteration(l, highlight=highlight, to_plot=to_plot_labels)

controls = [a, b, c, d, e, f, h, g, i, j, k, s_start, i_start, days, daily_fluctuations]
for w in controls:
    w.observe(update, names='value')

controls_panel = widgets.VBox(controls, layout={'width': '40%'})
fig_panel = widgets.Output()
selectors = []
for l in ['Suseptible', 'Infected', 'Removed']:
    selectors.append(widgets.Checkbox(True, description=l))
for l in ['Dead', 'Vaccinated', 'VI', 'FN-Tested']:
    selectors.append(widgets.Checkbox(False, description=l))
    
app = widgets.VBox([
    widgets.HBox(selectors),
    widgets.HBox([controls_panel, fig_panel])
])
display(app)

highlight = {'Infected': {
        'color': 'red',
        'linewidth': 3,
        'linestyle': '-.'
    },'Suseptible': {
        'color': 'green'
    }, 'Dead': {
        'color': 'black'
    }, 'VI': {
        'color': 'magenta'
}}
events = {
    200: {'a': 0.9, 'k': 0.1}, 
}
u